# Deplot notebook
> Creating an interactive environment for comparing stocks 



In [ ]:
#| default_exp deploy_project

In [ ]:
#| export
import buy_and_hold_vs_arima
import yfinance as yf
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Output, Input
import plotly.express as px
import pandas as pd
import numpy as np

In [ ]:
#help(buy_and_hold_vs_arima)
#buy_and_hold_vs_arima.foo("hba")
dir(buy_and_hold_vs_arima)
#dir(np)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__']

In [ ]:
#| export 


companies = ['AMZN','NFLX','GOOG']
tickers = [yf.Ticker(ticker).history( start='2021-12-10', end='2022-12-30')['High'].rename(ticker) for ticker in companies]

df = pd.concat(tickers, axis=1)
df

for a in df.columns:
    print(a)
    
df[['AMZN','GOOG']]

AMZN
NFLX
GOOG


,AMZN,GOOG
Date,,
2021-12-10 00:00:00-05:00,175.927002,149.399994
2021-12-13 00:00:00-05:00,172.100006,148.562500
2021-12-14 00:00:00-05:00,169.498993,145.442001
2021-12-15 00:00:00-05:00,173.600006,147.517242
2021-12-16 00:00:00-05:00,174.166000,148.551498
...,...,...
2022-12-22 00:00:00-05:00,85.680000,89.180000
2022-12-23 00:00:00-05:00,85.779999,90.099998
2022-12-27 00:00:00-05:00,85.349998,89.500000


In [ ]:
#| export

# http://127.0.0.1:8050/

#df = px.data.gapminder()
#df.columns



external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        dcc.Dropdown(
            id="dpdn2",
            value=["AMZN", "GOOG"],
            multi=True,
            options=[{"label": x, "value": x} for x in df.columns],
        ),
        html.Div(
            [
               # dcc.Graph(id="pie-graph", figure={}, className="six columns"),
                dcc.Graph(
                    id="my-graph",
                    figure={},
                    clickData=None,
                    hoverData=None,
                    config={
                        "staticPlot": False,  # True, False
                        "scrollZoom": True,  # True, False
                        "doubleClick": "reset",  # 'reset', 'autosize' or 'reset+autosize', False
                        "showTips": False,  # True, False
                        "displayModeBar": True,  # True, False, 'hover'
                        "watermark": True,
                        # 'modeBarButtonsToAdd': ['pan2d','select2d'],
                    },
                    className="six columns",
                ),
            ]
        ),
    ]
)


@app.callback(
    Output(component_id="my-graph", component_property="figure"),
    Input(component_id="dpdn2", component_property="value"),
)
def update_graph(country_chosen):
    #age_sex = titanic[["Age", "Sex"]]
    print(country_chosen)
    dff = df[country_chosen]
    fig = px.line(
        dff,
        x="Date[Days]",
        y="Stock",
        #color="country",
        #custom_data=["country", "continent", "lifeExp", "pop"],
    )
    fig.update_traces(mode="lines+markers")
    return fig







In [ ]:
#| export
#if __name__ == "__main__":
#    app.run_server(debug=False)


In [ ]:
import nbdev; nbdev.nbdev_export()